In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters 
num_input_neurons = 1000
num_network_neurons = 250
g_max = 0.02  
A_plus = 0.001  
A_minus = A_plus * 1.05  
tau_plus = 20  
tau_minus = 20  
simulation_time = 1000  # Reduce for testing
dt = 1  
stimulus_start, stimulus_end = 601, 800  
time_window = 100  

# LIF Parameters
C_m = 20  # Membrane capacitance (ms)
V_rest = -74  # Resting potential (mV)
V_thresh = -54  # Spiking threshold (mV)
V_reset = -60  # Reset potential (mV)
E_ex = 0  # Excitatory reversal potential (mV)
tau_ex = 5  # Synaptic decay constant (ms)

# Rate function parameters for input neurons
R0 = 10  
R1 = 80  
sigma = 100  

# Generate Corrected Spike Trains for Input Neurons (601-800)
def generate_corrected_spike_train(neuron_id, duration):
    spikes = []
    for t in range(duration):
        s = np.random.randint(stimulus_start, stimulus_end)  
        rate = R0 + R1 * (np.exp(-((s - neuron_id)**2) / (2 * sigma**2)) +
                          np.exp(-((s + 1000 - neuron_id)**2) / (2 * sigma**2)) +
                          np.exp(-((s - 1000 - neuron_id)**2) / (2 * sigma**2)))
        if np.random.rand() < (rate * dt / 1000):  
            spikes.append(t)
    return np.array(spikes)

input_spikes = {i: generate_corrected_spike_train(i, simulation_time) for i in range(stimulus_start, stimulus_end + 1)}

# Generate Poisson spike trains for network neurons
def generate_poisson_spike_train(rate, duration):
    return np.sort(np.random.uniform(0, duration, np.random.poisson(rate * (duration / 1000))))

network_spikes = {j: generate_poisson_spike_train(15, simulation_time) for j in range(num_network_neurons)}

# Initialize Feedforward Synaptic Strengths
feedforward_strengths = np.zeros((num_input_neurons, num_network_neurons))
feedforward_strengths[stimulus_start:stimulus_end+1, :] = np.random.uniform(0, g_max, (stimulus_end - stimulus_start + 1, num_network_neurons))
feedforward_strengths[:, 100:200] = 0  

# LIF Neuron Membrane Potentials
V = np.full(num_network_neurons, V_rest, dtype=np.float64)  
g_ex = np.zeros(num_network_neurons)  # Excitatory conductance

# STDP window function
def stdp_window(delta_t):
    return np.where(delta_t < 0, 
                    A_plus * np.exp(delta_t / tau_plus),  
                    -A_minus * np.exp(-delta_t / tau_minus))  

# Track spikes using a sparse dictionary
spike_times = {j: [] for j in range(num_network_neurons)}

# Precompute synaptic decay factor (avoids repeated `exp()` calls)
synaptic_decay = np.exp(-dt / tau_ex)

# Simulate LIF Neurons Over Time
for t in range(simulation_time):
    # Decay excitatory conductance
    g_ex *= synaptic_decay  

    # Compute feedforward inputs only for active neurons
    pre_spiking = np.zeros(num_input_neurons)
    for i in range(stimulus_start, stimulus_end + 1):  
        if t in input_spikes[i]:  
            pre_spiking[i] = 1  

    g_ex += feedforward_strengths.T @ pre_spiking  

    # Update membrane potential using LIF equation
    V += (V_rest - V + g_ex * (E_ex - V)) / C_m * dt  

    # Detect spiking neurons
    spiking_neurons = np.where(V >= V_thresh)[0]  
    for neuron in spiking_neurons:
        spike_times[neuron].append(t)  # Store only spike times (sparse)

    V[spiking_neurons] = V_reset  

    # Apply STDP only for neurons that spiked
    for j in spiking_neurons:
        for i in range(stimulus_start, stimulus_end + 1):  
            if t in input_spikes[i]:  
                delta_t = t - np.array(spike_times[j])  # Faster computation
                valid_times = delta_t[np.abs(delta_t) < time_window]  # Only valid pairs
                
                if valid_times.size > 0:  
                    weight_change = np.sum(stdp_window(valid_times))  
                    feedforward_strengths[i, j] += weight_change
                    feedforward_strengths[i, j] = np.clip(feedforward_strengths[i, j], 0, g_max)

# Normalize for Visualization
feedforward_strengths_normalized = feedforward_strengths / g_max

# PLOT 
plt.figure(figsize=(10, 6))

# Get the coordinates and strengths of non-zero synapses
y, x = np.where(feedforward_strengths_normalized > 0)  # y: input neurons, x: network neurons
weights = feedforward_strengths_normalized[y, x]  # Synaptic strengths
print(weights)

# Plot the scatter points
plt.scatter(x, y, c=weights, cmap='gray_r', s=10, marker='.', vmin=0, vmax=1)

# Add labels and title
plt.xlabel('Network Neuron')
plt.ylabel('Input Neuron')
plt.title('Feedforward Synaptic Strengths After STDP')

# Add colorbar
cbar = plt.colorbar(label='g/g_max')
cbar.set_ticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])

# Set axis limits
plt.xlim(0, 250)  # Network neurons
plt.ylim(0, 1000)  # Input neurons

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


#Load or Extract Feedforward Strengths from Figure 5A
feedforward_strengths_normalized = np.random.uniform(0, 1, (num_input_neurons, num_network_neurons))  # Placeholder

#Transfer Final Network Neuron Activity from 5A to 5B
recurrent_strengths = feedforward_strengths_normalized.T @ feedforward_strengths_normalized  # Compute recurrent connections
recurrent_strengths = recurrent_strengths / np.max(recurrent_strengths) * g_max  # Normalize


# Apply STDP on Transferred Recurrent Connections
for i in range(num_network_neurons):
    for j in range(num_network_neurons):
        if i != j and not (100 <= i < 200 and 100 <= j < 200):  # Prevent self-connections & enforce hole
            pre_spikes = np.array(network_spikes[i])
            post_spikes = np.array(network_spikes[j])

            if pre_spikes.size > 0 and post_spikes.size > 0:
                delta_t = pre_spikes[:, None] - post_spikes[None, :]
                mask = np.abs(delta_t) < time_window
                delta_t_windowed = delta_t[mask]

                if delta_t_windowed.size > 0:
                    updates = stdp_window(delta_t_windowed)
                    weight_change = np.sum(updates)

                    # **Ensure STDP does not modify w_ii**
                    if i != j:
                        recurrent_strengths[i, j] += weight_change
                        recurrent_strengths[i, j] = np.clip(recurrent_strengths[i, j], 0, g_max)


# **Enforce the hole again after STDP**
recurrent_strengths[100:200, 100:200] = 0  
# Same network neurons have no weight
recurrent_strengths[0:250, 0:100] = 0  
recurrent_strengths[0:250, 200:250] = 0  


# Normalize 
recurrent_strengths_normalized = recurrent_strengths / g_max

plt.figure(figsize=(10, 6))

# Get coordinates of non-zero recurrent synapses
y, x = np.where(recurrent_strengths_normalized > 0)
weights = recurrent_strengths_normalized[y, x]

plt.scatter(x, y, c=weights, cmap='gray_r', s=10, marker='.', vmin=0, vmax=1)

# Add labels and title
plt.xlabel('Postsynaptic Network Neuron')
plt.ylabel('Presynaptic Network Neuron')
plt.title('Figure 5B')

# Add colorbar
cbar = plt.colorbar(label='g/g_max')
cbar.set_ticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])

# Set axis limits
plt.xlim(0, 250)
plt.ylim(0, 250)

plt.show()
